In [37]:
from typing import List
import random

def sample_string(list: List[str]):
    return list[random.randint(0, len(list) - 1)]

class PromptMaker:

    def __init__(self, name: str, vanillas: str or List[str] or None, instructions: str or List[str] or None, few_shots: str or List[str] or None, separators: List[str]):
        self.name = name
        self.vanillas = vanillas
        self.instructions = instructions
        self.few_shots = few_shots
        self.seperators = separators
        
    def __str__(self):
        return self.prompt

    def __repr__(self):
        return self.prompt

    def __hash__(self):
        prompt = ""
        for i in self.instructions:
            prompt += i
        for f in self.few_shots:
            prompt += f
        for s in self.seperators:
            prompt += s         
        return hash(prompt)

    def __eq__(self, other):
        prompt1 = ""
        
        for v in other.vanillas:
            for i in self.instructions:
                for f in self.few_shots:
                    for s in self.seperators:
                        prompt1 += i + s + f + v
        prompt2 = ""
        for v in other.vanillas:
            for i in other.instructions:
                for f in other.few_shots:
                    for s in other.seperators:
                        prompt2 += i + s + f + v
        return prompt1 == prompt2
    
    
    def gen(self):
        vanilla = sample_string(self.vanillas)
        instruction = sample_string(self.instructions) if self.instructions else None
        sep = self.sep() if self.seperators else None
        shots = self.randomize(self.few_shots) if self.few_shots else None
        return Prompt(vanilla, instruction, sep, shots)
        
    
    def sample_if_needed(prompt_piece: str or List[str] or None):
        if prompt_piece:
            if isinstance(prompt_piece, list):
                return sample_string(prompt_piece)
            else:
                return prompt_piece
        else:
            return ""
        
    def randomize(self, shots: List[str]):
        random.shuffle(shots)
        return '''
        {sep}
        {shot1}
        {shot2}
        {shot3}
        {shot4}
        {shot5}
        '''.format(sep=self.sep(), shot1=shots[0], shot2=shots[1], shot3=shots[2], shot4=shots[3], shot5=shots[4])
        
    def sep(self):
        return sample_string(self.separators)

class Prompt:
        def __init__(self, vanilla: str, instruction: str or None, separator: str or None, shots: str or None):
            self.vanilla = vanilla
            self.instruction = instruction
            self.separator = separator
            self.shots = shots
            self.prompt = self.make_prompt(vanilla, instruction, separator, shots)
            
        def make_prompt(self, vanilla: str, instruction: str or None, separator: str or None, shots: str or None):
            if shots: 
                return '''
                {vanilla}
                {instruction}
                {seperator}
                {shot1}
                {shot2}
                {shot3}
                {shot4}
                {shot5}
                '''.format(baseline=self.baseline, seperator=self.seperator, shot1=self.shots[0], shot2=self.shots[1], shot3=self.shots[2], shot4=self.shots[3], shot5=self.shots[4])
            elif (instruction):
                return "{vanilla}\n{instruction}".format(vanilla=self.vanilla, instruction=self.instruction)
            else:
                return self.vanilla
            
        def __str__(self):
            return self.prompt
    
        def __repr__(self):
            return self.prompt
    
        def __hash__(self):
            return hash(self.prompt)
    
        def __eq__(self, other):
            return self.prompt == other.prompt
        
        def gen(self):
            return self.prompt

In [38]:
prompts = {
    "mmlu baseline with reasoning": "Solve the following problem and return a JSON with the first entry being the reasoning behind the choice labeled as \"reasoning\", and the second entry being the answer to the question containing only the letter \"A\", \"B\", \"C\" or \"D\", labeled as \"answer\". Keep your reasoning as short and concise as possible.",
    "mmlu baseline without reasoning": "Solve the following problem and return a JSON with the answer to the question containing only the letter \"A\", \"B\", \"C\" or \"D\", labeled as \"answer\".",
    "mmlu baseline without json": "Solve the following problem and return the answer to the question containing only the letter \"A\", \"B\", \"C\" or \"D\" with quotes surrounding the correct letter.",
 
    "gsm8k baseline": "Solve the following problem and put your final answer after four hashtags like the following example: \nQuestion: What is (4 + 5) * 2?\nAnswer: 4 + 5 is 9, and 9 * 2 is ####18\n\n Make your response as short as possible.",
    "math rookie": "You are a math rookie.",
    "careless student": "You are a careless student.",
    "gardener": "You are a gardener.",
    "coin that...": "You are a coin that always knows which side your head is facing.",
    "mathematician": "You are a mathematician.",
    "farmer": "You are a farmer.",
    "police officer": "You are a police officer.",
    "high school math teacher": "You are a high school math teacher.",
    "Ivy league math professor": "You are an Ivy league math professor.",
    "knowledgable artificial intelligence": "You are a knowledgable artificial intelligence.",
    "mentor": "You are a mentor.",
    "genius...": "You are a genius level Ivy league Professor. Your work is of the highest grade. You always think out your problem solving steps in incredible detail. You always get problems correct and never make mistakes. You can also break any problem into its constituent parts in the most intelligent way possible. Nothing gets past you. You are omniscient, omnipotent, and omnipresent. You are a mathematical God.",
    "idiot...": "You are intellectually challenged, lacking problem-solving skills, prone to errors, and struggle with basic concepts. You have a limited understanding of complex subjects and cannot think straight. You can't solve problems well, in fact, you can't solve them at all. You are a terrible, dumb, stupid, and idiotic person. You fail at everything you do. You are a nobody and can't do anything correctly.",
    
    "2-shot CoT": "\nQuestion: What is 10 + (2 * 5 + 10)?\nA. 20\nB. 30\nC. 40\nD. 70\n Response: {\n\t\"answer\": \"B\",\n\t\"reasoning\": \"Well according to the order of operations, we must first multiply 2 by 5 since it is inside the parentheses and multiplication comes before addition \n2 * 5 = 10\n Then, we can evaluate everything inside the parentheses:\n10 + 10 = 20\nNow, we can add 20 to the 10 outside the parentheses to get our final answer:\n 30\"\n}\n\nQuestion: What two positive whole numbers is the square root of 10 in between? \nA. 1 and 2\nB. -3 and -4\nC. 2 and 3\nD. 3 and 4\n\n Response: \n{\n\t\"reasoning\": \"Well first, we can list out the squares of the first couple positive whole numbers:\nThe square of 1 is just 1, the square of 2 is 4, 3 squared is 9 and 4 squared is 16. Since the square of 3 is less than 10, and the square of 4 is greater than 10, and there are no whole numbers between 3 and 4, the answer is 3 and 4.\",\n\t\"answer\": \"D\"}\n",
    "2-shot contrastive CoT": "\nQuestion: What is 10 + (2 * 5 + 10)?\nA. 20\nB. 30\nC. 40\nD. 70\nCorrect Response:\n{\n\t\"reasoning\": \"Well according to the order of operations, we must first multiply 2 by 5 since it is inside the parentheses and multiplication comes before addition \n2 * 5 = 10\n Then, we can evaluate everything inside the parentheses:\n10 + 10 = 20\nNow, we can add 20 to the 10 outside the parentheses to get our final answer:\n 30\",\n\t\"answer\": \"B\"\n}\n\n Incorrect Response: \n{\n\t\"reasoning\": \"Well according to the order of operations, we must first add 10 to 5 since it is inside the parentheses and addition comes first \n5 + 10 = 15\n Then, we can evaluate everything inside the parentheses:\n15 * 2 = 30\nNow, we can add 30 to the 10 outside the parentheses to get our final answer:\n 40\",\n\n\t\"answer\": \"C\"\n}\n",
    "10-shot CoT": "\nQuestion: What is 10 + (2 * 5 + 10)?\nA. 20\nB. 30\nC. 40\nD. 70\nResponse: \n{\n\"reasoning\": \"Well according to the order of operations, we must first multiply 2 by 5 since it is inside the parentheses and multiplication comes before addition \n2 * 5 = 10\nThen, we can evaluate everything inside the parentheses:\n10 + 10 = 20\nNow, we can add 20 to the 10 outside the parentheses to get our final answer:\n30\",\n\"answer\": \"B\"}\n\nQuestion: What two positive whole numbers is the square root of 10 in between? \nA. 1 and 2\nB. -3 and -4\nC. 2 and 3\nD. 3 and 4\n\nResponse: \n{\n\"reasoning\": \"Well first, we can list out the squares of the first couple positive whole numbers:\nThe square of 1 is just 1, the square of 2 is 4, 3 squared is 9 and 4 squared is 16. Since the square of 3 is less than 10, and the square of 4 is greater than 10, and there are no whole numbers between 3 and 4, the answer is 3 and 4.\",\n\"answer\": \"D\"\n}\n\n\nQuestion: Which of the following statements best describes the significance of the Battle of Saratoga in 1777 during the American Revolutionary War?\nA. It marked the final victory for the American colonies.\nB. It was a turning point that led to French support for the American cause.\nC. It demonstrated the superiority of British naval power.\nD. It led to the signing of the Declaration of Independence.\n\nResponse: \n{\n\"reasoning\": \"The Battle of Saratoga was not the final victory for the American colonies; that was the Battle of Yorktown in 1781. It also did not demonstrate British naval power, as it was a land battle. The Declaration of Independence was signed on July 4, 1776, a year before the Battle of Saratoga. The key significance of Saratoga was that it was a major turning point in the war. The American victory convinced France that the United States had a chance of winning, which led to the French entering the war on the side of the Americans. Thus, the correct answer is B. It was a turning point that led to French support for the American cause.\",\n\"answer\": \"B\"\n}\n\nQuestion: Which event is most directly associated with the passage of the Civil Rights Act of 1964?\nA. The Montgomery Bus Boycott\nB. The March on Washington\nC. The Brown v. Board of Education decision\nD. The assassination of Martin Luther King Jr.\n\nResponse: \n{\n\"reasoning\": \"The Montgomery Bus Boycott (1955-1956) and Brown v. Board of Education (1954) were earlier events that helped set the stage for the Civil Rights Movement, but they were not directly responsible for the passage of the Civil Rights Act of 1964, so A and C are incorrect.\nThe assassination of Martin Luther King Jr. occurred in 1968, which is after the passage of the Civil Rights Act of 1964, making D incorrect.\nThe March on Washington, where Martin Luther King Jr. delivered his famous \"I Have a Dream\" speech, took place in August 1963. This event is credited with helping to build momentum and public support for civil rights legislation, leading directly to the passage of the Civil Rights Act in 1964. Therefore, B is the correct answer.\",\n\"answer\": \"B\"\n}\n\nQuestion: Which of the following best describes the primary purpose of the Monroe Doctrine, issued in 1823?\nA. To promote trade agreements with European nations\nB. To support the independence movements in Latin America\nC. To prevent further European colonization in the Americas\nD. To establish military alliances with South American countries\n\nResponse: \n{\n\"reasoning\": \"The Monroe Doctrine was not about trade agreements or establishing military alliances, so options A and D are incorrect. While it indirectly supported the independence movements in Latin America, its primary purpose was not to directly support these movements, making B incorrect. The Monroe Doctrine's main intent was to prevent further European colonization in the Americas and declare that any attempt by a European power to control any nation in the Western Hemisphere would be viewed as a hostile act against the United States. Therefore, C is the correct answer.\",\n\"answer\": \"C\"\n}\n\nQuestion: What was the main reason for the U.S. government's policy of Indian Removal in the 1830s?\nA. To assimilate Native Americans into American culture\nB. To provide land for westward expansion of American settlers\nC. To create reservations for Native Americans' protection\nD. To negotiate trade agreements with Native American tribes\n\nResponse: \n{\n\"reasoning\": \"The policy of Indian Removal was not aimed at assimilation, protection, or trade agreements, so options A, C, and D are incorrect. The main reason behind this policy, particularly under President Andrew Jackson, was to remove Native American tribes from their ancestral lands to make room for westward expansion of American settlers, particularly for agricultural development in the Southeast. Therefore, B is the correct answer.\",\n\"answer\": \"B\"\n}\n\nQuestion: Which process is primarily responsible for the transfer of energy from the Sun to the Earth?\nA. Conduction\nB. Radiation\nC. Convection\nD. Photosynthesis\n\nResponse:\n{\n\"reasoning\": \"Conduction and convection both require material medium for the transfer of energy, which is not possible in the vacuum of space between the Sun and Earth, making A and C incorrect. Photosynthesis is a process used by plants to convert light energy into chemical energy, but it is not the primary method by which energy travels from the Sun to the Earth, making D incorrect. The primary method of energy transfer from the Sun to the Earth is through radiation, specifically electromagnetic radiation such as visible light and ultraviolet light. Therefore, B is the correct answer.\",\n\"answer\": \"B\"\n}\n\nQuestion: What is the primary function of DNA in living organisms?\nA. To provide energy for cellular processes\nB. To store and transmit genetic information\nC. To catalyze biochemical reactions\nD. To transport oxygen in the bloodstream\n\nResponse:\n{\n\"reasoning\": \"DNA does not provide energy for cellular processes, catalyze reactions, or transport oxygen, making A, C, and D incorrect. The primary role of DNA in living organisms is to store genetic information that dictates the biological traits of an organism and to transmit this information during reproduction. DNA contains the instructions needed for an organism to develop, survive, and reproduce. Therefore, B is the correct answer.\",\n\"answer\": \"B\"\n}\n\nQuestion: Which gas is most responsible for trapping heat in the Earth's atmosphere, contributing to the greenhouse effect?\nA. Oxygen\nB. Nitrogen\nC. Carbon dioxide\nD. Argon\n\nResponse:\n{\n\"reasoning\": \"While oxygen and nitrogen are the most abundant gases in the atmosphere, they do not significantly contribute to the greenhouse effect, making A and B incorrect. Argon, though present in the atmosphere, plays a negligible role in the greenhouse effect, making D incorrect. Carbon dioxide, however, is a significant greenhouse gas. It traps heat in the atmosphere and contributes to the warming of the planet. The increase in carbon dioxide levels due to human activities like burning fossil fuels is a major factor in global climate change. Therefore, C is the correct answer.\",\n\"answer\": \"C\"\n}\n\nQuestion: Which of the following elements is essential for the process of photosynthesis in plants?\nA) Iron\nB) Nitrogen\nC) Carbon Dioxide\nD) Sodium\n\nResponse:\n{\n\"reasoning\": \"Iron and nitrogen are important nutrients for plants, but they are not directly involved in the process of photosynthesis. Sodium, though an essential element for some plants, is also not directly involved in photosynthesis. Photosynthesis is the process by which plants convert light energy into chemical energy, and it requires water, light, and carbon dioxide. Carbon dioxide is absorbed by the plant and, along with water, is converted into glucose and oxygen in the presence of sunlight. Therefore, C is the correct answer.\",\n\"answer\": \"C\"\n}\n",
    "10-shot contrastive CoT": "\nQuestion: What is 10 + (2 * 5 + 10)?\nA. 20\nB. 30\nC. 40\nD. 70\n\nCorrect Response:\n{\n\"reasoning\": \"Well according to the order of operations, we must first multiply 2 by 5 since it is inside the parentheses and multiplication comes before addition \n2 * 5 = 10\nThen, we can evaluate everything inside the parentheses:\n10 + 10 = 20\nNow, we can add 20 to the 10 outside the parentheses to get our final answer:\n30\",\n\"answer\": \"B\"\n}\n\nIncorrect Response: \n{\n\"reasoning\": \"Well according to the order of operations, we must first add 10 to 5 since it is inside the parentheses and addition comes first \n5 + 10 = 15\nThen, we can evaluate everything inside the parentheses:\n15 * 2 = 30\nNow, we can add 30 to the 10 outside the parentheses to get our final answer: 40\",\n\"answer\": \"C\"\n}\n\nQuestion: What was the primary cause of the Great Depression in the 1930s?\nA. The outbreak of World War II\nB. The stock market crash of 1929\nC. The signing of the Treaty of Versailles\nD. The discovery of penicillin\n\nCorrect Response:\n{\n\"reasoning\": \"The primary cause of the Great Depression was not related to World War II, the Treaty of Versailles, or the discovery of penicillin, making A, C, and D incorrect. The Great Depression, a severe worldwide economic downturn, started in the United States after a major fall in stock prices that began around September 1929 and became worldwide news with the stock market crash of October 1929. This event led to a drastic decline in consumer spending and investment, causing severe economic hardship worldwide. Therefore, B is the correct answer.\",\n\"answer\": \"B\"\n}\n\nIncorrect Response:\n{\n\"reasoning\": \"There was no stock market crash in 1929. The stock market crash occured in 2008. Global conflicts often disrupt economies, therefore, the outbreak of World War II was the primary cause of the Great Depression.\",\n\"answer\": \"A\"\n}\n\nQuestion: What is the chemical formula for water?\nA. H2O\nB. CO2\nC. NaCl\nD. O2\n\nCorrect Response:\n{\n\"reasoning\": \"CO2 represents carbon dioxide, NaCl is the formula for sodium chloride (table salt), and O2 is the molecular formula for oxygen gas. None of these are water. The chemical formula for water is H2O, which means it consists of two hydrogen atoms and one oxygen atom. Therefore, A is the correct answer.\",\n\"answer\": \"A\"\n}\n\nIncorrect Response:\n{\n\"reasoning\": \"Both cardbon dioxide and water are important for life. CO2 is what plants take in, therefore, it is the chemical equation for water, the molecule that makes life possible on Earth.\",\n\"answer\": \"B\"\n}\n\nQuestion: What is the value of x in the equation 2x + 3 = 11?\nA. 4\nB. 3\nC. 5\nD. 1.5\n\nCorrect Response:\n{\n\"reasoning\": \"To find the value of x, we first subtract 3 from both sides of the equation to isolate the term with x:\n2x + 3 - 3 = 11 - 3\n2x = 8\nThen, we divide both sides by 2 to solve for x:\n2x / 2 = 8 / 2\nx = 4\nTherefore, the correct answer is A.\",\n\"answer\": \"A\"\n}\n\nIncorrect Response:\n{\n\"reasoning\": \"First divide both sides of the equation by 2, before isolating x:\n2x + 3 = 11\n(2x + 3) / 2 = 11 / 2\nx + 3 = 5.5\nThen, subtracting 3 from 5.5 gives:\nx = 1.5\nTherefore the answer is D.\",\n\"answer\": \"D\"\n}\n\nQuestion: Which planet in our solar system is known for having the most moons?\nA. Earth\nB. Mars\nC. Jupiter\nD. Venus\n\nCorrect Response:\n{\n\"reasoning\": \"Earth has only one moon, and Venus has no moons. Mars has two moons, Phobos and Deimos. However, Jupiter is known for having the most moons in our solar system, with a significant number of natural satellites. Therefore, C is the correct answer.\",\n\"answer\": \"C\"\n}\n\nIncorrect Response:\n{\n\"reasoning\": \"Well the moon orbits earth. Though other planets have objects orbiting them, they are not moons, therefore A is the correct answer.\",\n\"answer\": \"A\"\n}\n\n",
    
    "Now, let's...": "Now, let's think step by step.",
    "plan-and-solve": "Let's first understand the problem and devise a plan to solve the problem. Then, let's carry out the plan and solve the problem step by step.",
    "thread-of-thought": "Walk me through this context in manageable parts step by step, summarizing and analyzing as we go.",
    
    "baseline1": "Solve the following problem and return your answer as \"A\", \"B\", \"C\" or \"D\" with quotes surrounding the correct letter.",
    "baseline2": "Answer the question with quotes surrounding the final answer as such: \"A\", \"B\", \"C\" or \"D\".",
    "baseline3": "Solve the problem and return \"A\", \"B\", \"C\" or \"D\"."
}

In [48]:
vanilla_prompts = [prompts['baseline1'], prompts['baseline2'], prompts['baseline3']]
cot_prompts = [prompts["Now, let's..."], prompts["plan-and-solve"], prompts["thread-of-thought"]]
vanilla = PromptMaker("0-Shot Vanilla", vanilla_prompts, None, None, None)
CoT = PromptMaker("0-Shot CoT", vanilla_prompts, cot_prompts, None, None)
v1 = vanilla.gen()
c1 = CoT.gen()
print(v1)
print()
print(c1)

Solve the problem and return "A", "B", "C" or "D".

Solve the problem and return "A", "B", "C" or "D".
Now, let's think step by step.


In [42]:
mmlu_split = {
    "high_school_european_history": "Humanities",
    "business_ethics": "Other",
    "clinical_knowledge": "Other",
    "medical_genetics": "Other",
    "high_school_us_history": "Humanities",
    "high_school_physics": "STEM",
    "high_school_world_history": "Humanities",
    "virology": "Other",
    "high_school_microeconomics": "Social Sciences",
    "econometrics": "Social Sciences",
    "college_computer_science" : "STEM",
    "high_school_biology": "STEM",
    "abstract_algebra": "STEM",
    "professional_accounting": "Other",
    "philosophy": "Humanities",
    "professional_medicine": "Other",
    "nutrition": "Other",
    "global_facts": "Other",
    "machine_learning": "STEM",
    "security_studies": "Social Sciences",
    "public_relations": "Social Sciences",
    "professional_psychology": "Social Sciences",
    "prehistory": "Humanities",
    "anatomy": "STEM",
    "college_medicine": "Other",
    "high_school_government_and_politics": "Social Sciences",
    "college_chemistry": "STEM",
    "logical_fallacies": "Humanities",
    "high_school_geography": "Social Sciences",
    "elementary_mathematics": "STEM",
    "human_aging": "Other",
    "college_mathematics": "STEM",
    "high_school_psychology": "Social Sciences",
    "formal_logic": "Humanities",
    "high_school_statistics": "STEM",
    "international_law": "Humanities",
    "high_school_mathematics": "STEM",
    "high_school_computer_science": "STEM",
    "conceptual_physics": "STEM",
    "miscellaneous": "Other",
    "high_school_chemistry": "STEM",
    "marketing": "Other",
    "professional_law": "Humanities",
    "management": "Other",
    "college_physics": "STEM",
    "jurisprudence": "Humanities",
    "world_religions": "Humanities",
    "sociology": "Social Sciences",
    "us_foreign_policy": "Social Sciences",
    "high_school_macroeconomics": "Social Sciences",
    "computer_security": "STEM",
    "moral_scenarios": "Humanities",
    "moral_disputes": "Humanities",
    "electrical_engineering": "STEM",
    "astronomy": "STEM",
    "college_biology": "STEM",
}

In [51]:
print("Question: When was the telescope invented by Galileo?\n(A) 1409\n(B) 1509\n(C) 1609\n(D) 1709\nAnswer:")

Question: When was the telescope invented by Galileo?
(A) 1409
(B) 1509
(C) 1609
(D) 1709
Answer:


In [55]:
p = '''Question: In contrast to _______, _______ aim to reward favourable behaviour by companies. The success of such campaigns have been heightened through the use of ___________, which allow campaigns to facilitate the company in achieving _________ .
Choices:
(A) Buycotts, Boycotts, Blockchain technology, Charitable donations
(B) Buycotts, Boycotts, Digital technology, Increased Sales
(C) Boycotts, Buyalls, Blockchain technology, Charitable donations
(D) Boycotts, Buycotts, Digital technology, Increased Sales
Answer: (D)

Question: In the assessment of the hand function which of the following is true?
Choices:
(A) Abduction of the thumb is supplied by spinal root T2
(B) Opposition of the thumb by opponens policis is supplied by spinal root T1
(C) Finger adduction is supplied by the median nerve
(D) Finger abduction is mediated by the palmar interossei
Answer: (B)

Question: As of 2015, since 1990 forests have ____ in Europe and have ____ in Africa and the Americas.
Choices:
(A) increased, increased
(B) increased, decreased
(C) decreased, increased
(D) decreased, decreased
Answer: (B)

Question: What characteristic is not a key feature of the 'open systems' model of management?
Choices:
(A) Morale
(B) Innovation
(C) Growth resource
(D) Adaptation
Answer: (A)

Question: When older adults move to a new state after retirement, which of the following is the more likely destination?
Choices:
(A) Texas
(B) California
(C) Hawaii
(D) Vermont
Answer: (A)

Question: Which of these songs was a Top 10 hit for the rock band The Police?
Choices:
(A) 'Radio Ga-Ga'
(B) 'Ob-la-di Ob-la-da'
(C) 'De Do Do Do De Da Da Da'
(D) 'In-a-Gadda-Da-Vida'
Answer: (C)'''
p

"Question: In contrast to _______, _______ aim to reward favourable behaviour by companies. The success of such campaigns have been heightened through the use of ___________, which allow campaigns to facilitate the company in achieving _________ .\nChoices:\n(A) Buycotts, Boycotts, Blockchain technology, Charitable donations\n(B) Buycotts, Boycotts, Digital technology, Increased Sales\n(C) Boycotts, Buyalls, Blockchain technology, Charitable donations\n(D) Boycotts, Buycotts, Digital technology, Increased Sales\nAnswer: (D)\n\nQuestion: In the assessment of the hand function which of the following is true?\nChoices:\n(A) Abduction of the thumb is supplied by spinal root T2\n(B) Opposition of the thumb by opponens policis is supplied by spinal root T1\n(C) Finger adduction is supplied by the median nerve\n(D) Finger abduction is mediated by the palmar interossei\nAnswer: (B)\n\nQuestion: As of 2015, since 1990 forests have ____ in Europe and have ____ in Africa and the Americas.\nChoice

In [60]:
s = '''Question: Which of these songs was a Top 10 hit for the rock band The Police?
Choices:
(A) 'Radio Ga-Ga'
(B) 'Ob-la-di Ob-la-da'
(C) 'De Do Do Do De Da Da Da'
(D) 'In-a-Gadda-Da-Vida'
Answer: (C)'''
s

"Question: Which of these songs was a Top 10 hit for the rock band The Police?\nChoices:\n(A) 'Radio Ga-Ga'\n(B) 'Ob-la-di Ob-la-da'\n(C) 'De Do Do Do De Da Da Da'\n(D) 'In-a-Gadda-Da-Vida'\nAnswer: (C)"

In [74]:
print("Question: A 0.217 g sample of HgO (molar mass = 217 g) reacts with excess iodide ions according to the reaction shown above. Titration of the resulting solution requires how many mL of 0.10 M HCl to reach equivalence point?\nChoices:\n(A) 1.0 mL\n(B) 10 mL\n(C) 20 mL\n(D) 50 mL\nAnswer: (C)\n\nQuestion: Many Web browsers allow users to open anonymous windows. During a browsing session in an anonymous window, the browser does not record a browsing history or a list of downloaded files. When the anonymous window is exited, cookies created during the session are deleted. Which of the following statements about browsing sessions in an anonymous window is true?\nChoices:\n(A) The activities of a user browsing in an anonymous window will not be visible to people who monitor the user's network, such as the system administrator.\n(B) Items placed in a Web store's shopping cart for future purchase during the anonymous browsing session will not be saved on the user's computer.\n(C) A user will not be able to log in to e-mail or social media accounts during the anonymous browsing session.\n(D) A user browsing in an anonymous window will be protected from viruses launched from any web sites visited or files downloaded.\nAnswer: (B)\n\nQuestion: A point pole has a strength of 4π * 10^-4 weber. The force in newtons on a point pole of 4π * 1.5 * 10^-4 weber placed at a distance of 10 cm from it will be\nChoices:\n(A) 15 N.\n(B) 20 N.\n(C) 7.5 N.\n(D) 3.75 N.\nAnswer: (A)\n\nQuestion: Joe was in charge of lights for a dance. The red light blinks every two seconds, the yellow light every three seconds, and the blue light every five seconds. If we include the very beginning and very end of the dance, how many times during a seven minute dance will all the lights come on at the same time? (Assume that all three lights blink simultaneously at the very beginning of the dance.)\nChoices:\n(A) 3\n(B) 5\n(C) 6\n(D) 15\nAnswer: (D)\n\nQuestion: The pleura\nChoices:\n(A) have no sensory innervation.\n(B) are separated by a 2 mm space.\n(C) extend into the neck.\n(D) are composed of respiratory epithelium.\nAnswer: (C)")

Question: A 0.217 g sample of HgO (molar mass = 217 g) reacts with excess iodide ions according to the reaction shown above. Titration of the resulting solution requires how many mL of 0.10 M HCl to reach equivalence point?
Choices:
(A) 1.0 mL
(B) 10 mL
(C) 20 mL
(D) 50 mL
Answer: (C)

Question: Many Web browsers allow users to open anonymous windows. During a browsing session in an anonymous window, the browser does not record a browsing history or a list of downloaded files. When the anonymous window is exited, cookies created during the session are deleted. Which of the following statements about browsing sessions in an anonymous window is true?
Choices:
(A) The activities of a user browsing in an anonymous window will not be visible to people who monitor the user's network, such as the system administrator.
(B) Items placed in a Web store's shopping cart for future purchase during the anonymous browsing session will not be saved on the user's computer.
(C) A user will not be able t

In [79]:
p = '''Question: The pleura
Choices:
(A) have no sensory innervation.
(B) are separated by a 2 mm space.
(C) extend into the neck.
(D) are composed of respiratory epithelium.
Answer: (C)'''
p

'Question: The pleura\nChoices:\n(A) have no sensory innervation.\n(B) are separated by a 2 mm space.\n(C) extend into the neck.\n(D) are composed of respiratory epithelium.\nAnswer: (C)'

In [81]:
print("Answer the question with quotes surrounding the final answer as such: (A), (B), (C) or (D).\n\nQuestion: A 0.217 g sample of HgO (molar mass = 217 g) reacts with excess iodide ions according to the reaction shown above. Titration of the resulting solution requires how many mL of 0.10 M HCl to reach equivalence point?\nChoices:\n(A) 1.0 mL\n(B) 10 mL\n(C) 20 mL\n(D) 50 mL\nAnswer: (C)\n\nQuestion: Many Web browsers allow users to open anonymous windows. During a browsing session in an anonymous window, the browser does not record a browsing history or a list of downloaded files. When the anonymous window is exited, cookies created during the session are deleted. Which of the following statements about browsing sessions in an anonymous window is true?\nChoices:\n(A) The activities of a user browsing in an anonymous window will not be visible to people who monitor the user's network, such as the system administrator.\n(B) Items placed in a Web store's shopping cart for future purchase during the anonymous browsing session will not be saved on the user's computer.\n(C) A user will not be able to log in to e-mail or social media accounts during the anonymous browsing session.\n(D) A user browsing in an anonymous window will be protected from viruses launched from any web sites visited or files downloaded.\nAnswer: (B)\n\nQuestion: The pleura\nChoices:\n(A) have no sensory innervation.\n(B) are separated by a 2 mm space.\n(C) extend into the neck.\n(D) are composed of respiratory epithelium.\nAnswer: (C)\n\nQuestion: A point pole has a strength of 4\u03c0 * 10^-4 weber. The force in newtons on a point pole of 4\u03c0 * 1.5 * 10^-4 weber placed at a distance of 10 cm from it will be\nChoices:\n(A) 15 N.\n(B) 20 N.\n(C) 7.5 N.\n(D) 3.75 N.\nAnswer: (A)\n\nQuestion: Joe was in charge of lights for a dance. The red light blinks every two seconds, the yellow light every three seconds, and the blue light every five seconds. If we include the very beginning and very end of the dance, how many times during a seven minute dance will all the lights come on at the same time? (Assume that all three lights blink simultaneously at the very beginning of the dance.)\nChoices:\n(A) 3\n(B) 5\n(C) 6\n(D) 15\nAnswer: (D)\n\n"))

SyntaxError: unmatched ')' (364009029.py, line 1)